In [1]:
!pip install pandas transformers torch scikit-learn

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [2]:
!pip install pytorch-accelerated[examples]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset

In [4]:
fields = ['text', 'sentiment']
df = pd.read_csv('train.csv',
    encoding='ISO-8859-1',
    on_bad_lines='skip',
    # delimiter=';',
    usecols=fields
)
df.head(5)

,text,sentiment
0,"I`d have responded, if I were going",neutral
1,Sooo SAD I will miss you here in San Diego!!!,negative
2,my boss is bullying me...,negative
3,what interview! leave me alone,negative
4,"Sons of ****, why couldn`t they put them on t...",negative


In [5]:
# NaN 값 제거
df = df.dropna()

# 샘플링 (예: 1000개의 샘플만 사용)
df = df.sample(n=2000, replace=True, random_state=42)

# 레이블 인코딩
label_dict = {'neutral': 0, 'positive': 1, 'negative': 2}
df['label'] = df['sentiment'].map(label_dict)

# 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)


In [6]:
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }


In [7]:
# 토크나이저 초기화
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [8]:
# 데이터셋 준비
train_dataset = SentimentDataset(X_train.tolist(), y_train.tolist(), tokenizer, max_len=50)
test_dataset = SentimentDataset(X_test.tolist(), y_test.tolist(), tokenizer, max_len=50)


In [9]:
# 모델 초기화
model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=3)
model.config.pad_token_id = model.config.eos_token_id

# 트레이닝 인자 설정
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    warmup_steps=10,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
)

# 트레이너 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# 모델 학습
trainer.train()

# 평가
results = trainer.evaluate()
print(f"Evaluation results: {results}")

# 예측
predictions = trainer.predict(test_dataset)
preds = predictions.predictions.argmax(-1)
print(f"Predictions: {preds}")


Epoch,Training Loss,Validation Loss
1,0.964300,0.929366


Evaluation results: {'eval_loss': 0.9293659925460815, 'eval_runtime': 2.6563, 'eval_samples_per_second': 150.584, 'eval_steps_per_second': 75.292, 'epoch': 1.0}
Predictions: [2 0 2 0 1 1 2 0 1 2 0 2 1 1 1 2 1 1 2 2 0 0 1 0 2 1 0 1 2 1 2 2 2 2 0 2 2
 2 1 0 1 2 2 2 1 1 2 0 1 0 1 1 1 0 0 2 2 2 0 1 0 1 2 1 0 2 0 0 0 0 1 0 2 0
 2 2 1 2 1 1 0 2 2 1 2 1 1 0 2 1 0 0 0 1 2 2 1 2 0 1 0 1 2 0 1 2 2 0 1 2 1
 1 2 1 0 2 2 1 1 0 0 0 0 2 2 2 2 2 1 0 1 1 1 0 0 2 0 0 2 2 0 0 2 1 0 1 0 1
 2 1 0 1 1 1 2 2 1 2 2 1 0 1 2 0 0 2 1 0 2 2 0 2 2 1 2 1 1 0 2 1 0 0 1 0 1
 0 0 1 0 2 2 2 1 1 2 1 0 1 2 2 2 0 0 0 1 2 1 0 2 0 1 0 1 1 2 1 2 0 0 1 2 2
 0 1 2 0 2 0 2 1 2 1 1 2 0 1 2 2 0 1 1 1 1 1 0 1 0 1 0 0 1 0 2 0 2 0 0 1 1
 1 1 1 1 0 0 1 1 2 1 1 1 0 2 0 2 1 2 1 0 0 0 1 2 1 2 2 0 1 2 0 1 0 2 2 2 2
 1 0 1 2 2 0 1 0 2 2 2 2 0 0 2 1 0 1 0 2 1 0 1 0 2 1 1 0 1 1 2 1 1 1 0 0 1
 2 2 0 1 2 0 0 2 0 0 0 1 0 2 0 2 0 0 2 2 1 2 0 0 2 2 2 1 0 2 0 1 2 1 0 0 0
 1 1 0 0 0 1 1 1 1 1 1 1 2 2 1 2 1 0 2 0 1 2 0 1 0 1 2 0 0 1]


In [11]:

# 평가
results = trainer.evaluate()
print(f"Evaluation results: {results}")

# 테스트 데이터로 예측 수행
def predict(model, tokenizer, dataset):
    predictions = trainer.predict(dataset)
    preds = predictions.predictions.argmax(-1)
    return preds

test_preds = predict(model, tokenizer, test_dataset)
print(f"Test Predictions: {test_preds}")


Evaluation results: {'eval_loss': 0.9293659925460815, 'eval_runtime': 7.7866, 'eval_samples_per_second': 51.37, 'eval_steps_per_second': 25.685, 'epoch': 1.0}
Test Predictions: [2 0 2 0 1 1 2 0 1 2 0 2 1 1 1 2 1 1 2 2 0 0 1 0 2 1 0 1 2 1 2 2 2 2 0 2 2
 2 1 0 1 2 2 2 1 1 2 0 1 0 1 1 1 0 0 2 2 2 0 1 0 1 2 1 0 2 0 0 0 0 1 0 2 0
 2 2 1 2 1 1 0 2 2 1 2 1 1 0 2 1 0 0 0 1 2 2 1 2 0 1 0 1 2 0 1 2 2 0 1 2 1
 1 2 1 0 2 2 1 1 0 0 0 0 2 2 2 2 2 1 0 1 1 1 0 0 2 0 0 2 2 0 0 2 1 0 1 0 1
 2 1 0 1 1 1 2 2 1 2 2 1 0 1 2 0 0 2 1 0 2 2 0 2 2 1 2 1 1 0 2 1 0 0 1 0 1
 0 0 1 0 2 2 2 1 1 2 1 0 1 2 2 2 0 0 0 1 2 1 0 2 0 1 0 1 1 2 1 2 0 0 1 2 2
 0 1 2 0 2 0 2 1 2 1 1 2 0 1 2 2 0 1 1 1 1 1 0 1 0 1 0 0 1 0 2 0 2 0 0 1 1
 1 1 1 1 0 0 1 1 2 1 1 1 0 2 0 2 1 2 1 0 0 0 1 2 1 2 2 0 1 2 0 1 0 2 2 2 2
 1 0 1 2 2 0 1 0 2 2 2 2 0 0 2 1 0 1 0 2 1 0 1 0 2 1 1 0 1 1 2 1 1 1 0 0 1
 2 2 0 1 2 0 0 2 0 0 0 1 0 2 0 2 0 0 2 2 1 2 0 0 2 2 2 1 0 2 0 1 2 1 0 0 0
 1 1 0 0 0 1 1 1 1 1 1 1 2 2 1 2 1 0 2 0 1 2 0 1 0 1 2 0 0 1]


In [13]:
from sklearn.metrics import accuracy_score

# 정확도 계산
accuracy = accuracy_score(y_test, test_preds)
print(f"Test Accuracy: {accuracy}")


Test Accuracy: 0.76
